# Simple PyTorch Example

Train a small neural network to approximate **y = 2x + 1** on synthetic data.

**Python 3.11** recommended.

---

## Cloudera AI Workbench (important)

If you're on **Cloudera AI Workbench**, installing PyTorch inside a notebook can crash the kernel.

1. **In a terminal** (not in this notebook), run:
   ```bash
   pip install --no-cache-dir torch --index-url https://download.pytorch.org/whl/cpu
   ```
   Or from the project folder: `pip install --no-cache-dir -r requirements-workbench.txt`

2. **Restart the session** (or start a new one).

3. Then open this notebook and run the cells below. Do **not** run the install cell in this notebook on Workbench—use the terminal and restart instead.

On other environments (local, Colab, etc.), you can run the install cell below if PyTorch isn't already installed.

## 1. Install PyTorch (if needed)

Run this cell only if PyTorch is not installed. **On Cloudera AI Workbench**, use the terminal + restart steps above instead of this cell.

In [ ]:
# CPU-only install (smaller, avoids CUDA). Skip on Cloudera AI Workbench—use terminal + restart.
%pip install --no-cache-dir torch --index-url https://download.pytorch.org/whl/cpu

## 2. Imports and data

Synthetic data: **y = 2x + 1** plus small noise.

In [ ]:
import torch
import torch.nn as nn

# Create synthetic data: y = 2x + 1 + small noise
torch.manual_seed(42)
x = torch.randn(100, 1) * 3  # 100 points in roughly [-9, 9]
y = 2 * x + 1 + 0.1 * torch.randn(100, 1)

print(f"Data: {x.shape[0]} points. Target: y ≈ 2x + 1")

## 3. Model

Small MLP: input → 16 units (ReLU) → output.

In [ ]:
class SimpleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(1, 16),
            nn.ReLU(),
            nn.Linear(16, 1),
        )

    def forward(self, x):
        return self.layers(x)

model = SimpleNet()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
print(model)

## 4. Train

200 epochs, MSE loss, Adam.

In [ ]:
model.train()
for epoch in range(200):
    optimizer.zero_grad()
    pred = model(x)
    loss = criterion(pred, y)
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 50 == 0:
        print(f"Epoch {epoch + 1}, Loss: {loss.item():.4f}")

## 5. Check prediction

At **x = 1**, we expect **y ≈ 3**.

In [ ]:
model.eval()
with torch.no_grad():
    test_x = torch.tensor([[1.0]])
    pred_y = model(test_x)
    print(f"At x=1, predicted y ≈ {pred_y.item():.2f} (expected ≈ 3)")